In [14]:
import requests
import pprint
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import re
import random


In [15]:
data = pd.read_csv("article_links.csv")

In [16]:
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

def get_article(url):
    page = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    soup  = BeautifulSoup(page.content, 'html.parser')
    article = soup.find_all('p')
    return article

In [ ]:
index = 0
l= []
for url in data['0'].values:
    index += 1
    if index % 10 == 0:
        print(index)
    l.append(get_article(url))

In [ ]:
# Function to scrape an individual article
def scrapeArticle(link):
    parsed_page = requestPage(link)
    if parsed_page:
        title_element = parsed_page.find('h1', class_='article-heading__title')
        if title_element:
            title = title_element.text.strip()
        else:
            title = "No Title"

        author_element = parsed_page.find('a', class_='author-link')
        if author_element:
            author = author_element.text.strip()
        else:
            author = "Unknown"

        date_element = parsed_page.find('time', class_='date__time')
        if date_element and 'datetime' in date_element.attrs:
            date = date_element['datetime']
        else:
            date = "Unknown"

        content_element = parsed_page.find('div', class_='wysiwyg')
        if content_element:
            paragraphs = [p.text.strip() for p in content_element.find_all('p')]
            content = '\n'.join(paragraphs)
        else:
            content = "No Content"
        
        # Create a dictionary to store the scraped data
        article_data = {
            'Title': title,
            'Author': author,
            'Date': date,
            'Link': link,
            'Content': content
        }
        
        return article_data
    else:
        print(f"Failed to scrape article: {link}")
        return None


In [32]:
def extract_date_from_url(url):
    # Define the regular expression pattern to match the date format in the URL
    date_pattern = r'(\d{4})/(\d{1,2})/(\d{1,2})/'
    
    # Search for the date pattern in the URL
    match = re.search(date_pattern, url)
    
    if match:
        # Extract year, month, and day from the matched groups
        year = int(match.group(1))
        month = int(match.group(2))
        day = int(match.group(3))
        
        return pd.to_datetime(f'{year}-{month}-{day}').date()
    else:
        return None

# Example usage:
data['date'] = [extract_date_from_url(url) for url in data['0'].values]

In [34]:
data.to_csv("article_links_updated.csv", index=False)

In [38]:
def extract_text(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(str(html_content), "html.parser")
    # Extract and return the textual part of the HTML
    return soup.get_text(separator="\n")

In [39]:
data['text_up'] = [extract_text(article) for article in data['article'].values]

In [42]:
data['text_up'] = data['text_up'].str.replace('\n', ' ')

In [45]:
data.to_csv("article_links_updated.csv", index=False)